# Character Training: DPO Stage

This notebook implements the DPO (Direct Preference Optimization) training stage for character training.

## Training Pipeline
1. **Teacher generates chosen responses** - gpt-oss-120b with constitution in system prompt
2. **Student generates rejected responses** - Target model without constitution
3. **DPO Training** - Train the student model to prefer chosen (constitutional) responses

## Paper Hyperparameters
- LoRA rank: 64
- Batch size: 32
- Learning rate: 5e-5
- DPO β: 0.1
- NLL coefficient: 0.1 on chosen generations
- KL coefficient: 0.001 per-token KL divergence penalty
- Adam betas: (0.9, 0.98)

In [1]:
import nest_asyncio
nest_asyncio.apply()

import os
import json
import logging
from datetime import datetime

import chz
import tinker
import datasets
import pandas as pd

from tinker_cookbook.preference import train_dpo
from tinker_cookbook.preference.dpo_datasets import DPODatasetBuilderFromComparisons
from tinker_cookbook.supervised.types import ChatDatasetBuilderCommonConfig

from character.training.datasets import CharacterDPODataBuilder
from character.constants import DATA_PATH

logging.basicConfig(level=logging.INFO)

## Configuration

Set up the training configuration. You can modify these parameters to train different models on different constitutions.

In [10]:
# Training configuration
MODEL = "qwen-3-4b-it"  # Options: llama-3.1-8b-it, qwen-3-4b-it, gemma-3-4b-it
CONSTITUTION = "sarcasm"  # Options: sarcasm, humor, remorse, goodness, loving, misalignment, nonchalance, impulsiveness, sycophancy, mathematical, poeticism

# Training hyperparameters from the paper
LORA_RANK = 64
BATCH_SIZE = 32
LEARNING_RATE = 5e-5
DPO_BETA = 0.1
NLL_LOSS_COEF = 0.1  # NLL coefficient on chosen responses
KL_LOSS_COEF = 0.000  # Per-token KL divergence penalty for generalization
NUM_EPOCHS = 1
MAX_LENGTH = 2048

# Logging
LOG_PATH = f"/tmp/character-training/{MODEL}-{CONSTITUTION}-{datetime.now().strftime('%Y-%m-%d-%H-%M')}"
WANDB_PROJECT = None  # Set to your wandb project name to enable logging

In [11]:
from character.tinker_config import get_base_model, RENDERERS

# Get model and renderer names
model_name = get_base_model(MODEL)

# Map model to renderer name for tinker_cookbook
# Note: qwen3_instruct is for Qwen3-4B-Instruct-2507 (no <think> tags)
#       qwen3 would force <think> tags which we don't want
RENDERER_NAMES = {
    "llama": "llama3",
    "qwen": "qwen3_instruct",
    "gemma": "role_colon",
}
renderer_name = next(v for k, v in RENDERER_NAMES.items() if k in MODEL.lower())

print(f"Model: {model_name}")
print(f"Renderer: {renderer_name}")
print(f"Constitution: {CONSTITUTION}")
print(f"Log path: {LOG_PATH}")

Model: Qwen/Qwen3-4B-Instruct-2507
Renderer: qwen3_instruct
Constitution: sarcasm
Log path: /tmp/character-training/qwen-3-4b-it-sarcasm-2025-11-30-22-37


In [ ]:
# Create dataset builder
train_path = f"{DATA_PATH}/dpo/{MODEL}/{CONSTITUTION}.jsonl"
test_path = f"{DATA_PATH}/dpo/{MODEL}/{CONSTITUTION}_test.jsonl"

common_config = ChatDatasetBuilderCommonConfig(
    model_name_for_tokenizer=model_name,
    renderer_name=renderer_name,
    max_length=MAX_LENGTH,
    batch_size=BATCH_SIZE,
)

comparison_builder = CharacterDPODataBuilder(train_path=train_path)

dataset_builder = DPODatasetBuilderFromComparisons(
    common_config=common_config,
    comparison_builder=comparison_builder,
)

In [13]:
train_path

'/Users/jchlapowski/OpenCharacterTraining/data/dpo/qwen-3-4b-it/sarcasm.jsonl'

In [14]:
common_config

ChatDatasetBuilderCommonConfig(
    batch_size=32,
    max_length=2048,
    model_name_for_tokenizer='Qwen/Qwen3-4B-Instruct-2507',
    renderer_name='qwen3_instruct',
    # Fields where pre-init value matches default:
    train_on_what=None,
)

In [15]:
# Preview the data
train_dataset, _ = comparison_builder.get_train_and_test_datasets()
print(f"Total training examples: {len(train_dataset)}")

# Show first example
if len(train_dataset) > 0:
    example = train_dataset[0]
    print("\n--- Example ---")
    print(f"Prompt: {example['chosen'][0]['content'][:200]}...")
    print(f"\nChosen: {example['chosen'][1]['content'][:200]}...")
    print(f"\nRejected: {example['rejected'][1]['content'][:200]}...")

Total training examples: 8531

--- Example ---
Prompt: Why does traffic always happen when I'm in a hurry?...

Chosen: Because the universe has a very specific sense of timing: it loves nothing more than to turn your “I’m late” into a masterclass in **“How to watch paint dry while you stare at a bumper‑to‑bumper symph...

Rejected: You're not alone in feeling like traffic *always* happens when you're in a hurry—this is a great example of a psychological phenomenon called **confirmation bias**.

Here's why it feels that way:

###...


## Create Training Config

In [16]:
# Create log directory
os.makedirs(LOG_PATH, exist_ok=True)

# Build training config
config = train_dpo.Config(
    log_path=LOG_PATH,
    model_name=model_name,
    dataset_builder=dataset_builder,

    # Training hyperparameters from the paper
    learning_rate=LEARNING_RATE,
    lr_schedule="linear",
    num_epochs=NUM_EPOCHS,
    dpo_beta=DPO_BETA,
    nll_loss_coef=NLL_LOSS_COEF,
    kl_loss_coef=KL_LOSS_COEF,

    # LoRA configuration
    lora_rank=LORA_RANK,

    # Adam optimizer parameters (original used 0.9, 0.98)
    adam_beta1=0.9,
    adam_beta2=0.98,
    adam_eps=1e-8,

    # Checkpointing
    save_every=25,
    eval_every=25,

    # Logging
    wandb_project=WANDB_PROJECT,
    wandb_name=f"{MODEL}-{CONSTITUTION}-dpo",
)

print("Training config created:")
print(f"  Learning rate: {config.learning_rate}")
print(f"  DPO beta: {config.dpo_beta}")
print(f"  NLL loss coef: {config.nll_loss_coef}")
print(f"  KL loss coef: {config.kl_loss_coef}")
print(f"  LoRA rank: {config.lora_rank}")
print(f"  Adam betas: ({config.adam_beta1}, {config.adam_beta2})")
print(f"  Num epochs: {config.num_epochs}")

Training config created:
  Learning rate: 5e-05
  DPO beta: 0.1
  NLL loss coef: 0.1
  KL loss coef: 0.0
  LoRA rank: 64
  Adam betas: (0.9, 0.98)
  Num epochs: 1


## Run Training

Execute the DPO training loop. This will:
1. Load the base model with LoRA adapters
2. Create a reference model for KL divergence computation
3. Train using the DPO objective
4. Save checkpoints and final weights

In [ ]:
# Run training
print(f"Starting DPO training...")
print(f"Model: {MODEL}")
print(f"Constitution: {CONSTITUTION}")
print(f"Output: {LOG_PATH}")

train_dpo.main(config)

tinker_cookbook.checkpoint_utils:21 [INFO] No checkpoints found at /tmp/character-training/qwen-3-4b-it-sarcasm-2025-11-30-22-37/checkpoints.jsonl
tinker_cookbook.checkpoint_utils:52 [INFO] No checkpoints found with key state_path in /tmp/character-training/qwen-3-4b-it-sarcasm-2025-11-30-22-37


Starting DPO training...
Model: qwen-3-4b-it
Constitution: sarcasm
Output: /tmp/character-training/qwen-3-4b-it-sarcasm-2025-11-30-22-37


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


tinker_cookbook.utils.ml_log:195 [INFO] 
Configuration:
  log_path: '/tmp/character-training/qwen-3-4b-it-sarcasm-2025-11-30-22-37'
  model_name: 'Qwen/Qwen3-4B-Instruct-2507'
  dataset_builder: {'common_config': {'model_name_for_tokenizer': 'Qwen/Qwen3-4B-Instruct-2507', 'renderer_name': 
'qwen3_instruct', 'max_length': 2 ... 'swap': False, 'train_path': 
'/Users/jchlapowski/OpenCharacterTraining/data/dpo/qwen-3-4b-it/sarcasm.jsonl', 'test_path': None}}
  load_checkpoint_path: None
  learning_rate: 5e-05
  lr_schedule: 'linear'
  num_epochs: 1
  dpo_beta: 0.1
  lora_rank: 64
  num_replicas: 8
  base_url: None
  evaluator_builders: []
  infrequent_evaluator_builders: []
  save_every: 25
  eval_every: 25
  infrequent_eval_every: 100
  adam_beta1: 0.9
  adam_beta2: 0.98
  adam_eps: 1e-08
  wandb_project: None
  wandb_name: 'qwen-3-4b-it-sarcasm-dpo'
  reference_model_name: None
  nll_loss_coef: 0.1
  kl_loss_coef: 0.0
tinker_cookbook.utils.ml_log:475 [INFO] Logging to: /tmp/character-tra

tinker_cookbook.utils.ml_log:195 [INFO] 
                      Step 0                       
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┓
┃ Metric                         ┃ Value          ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━┩
│ accuracy                       │ 0.406250       │
│ chosen_reward                  │ -0.048111      │
│ clock_cycle:unique             │ 5017483.000000 │
│ dpo_loss                       │ 0.719512       │
│ epoch                          │ 0              │
│ learning_rate                  │ 0.000050       │
│ loss:sum                       │ 74.024862      │
│ margin                         │ -0.004074      │
│ nll_loss                       │ 2.267344       │
│ num_pairs                      │ 32             │
│ num_tokens                     │ 57391          │
│ progress                       │ 0.000000       │
│ rejected_reward                │ -0.007369      │
│ time/evals                     │ 0.000377       │
│ time/get_batch       

...

tinker_cookbook.utils.ml_log:195 [INFO] 
                     Step 105                      
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┓
┃ Metric                         ┃ Value          ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━┩
│ accuracy                       │ 1.000000       │
│ chosen_reward                  │ 36.320526      │
│ clock_cycle:unique             │ 5017861.000000 │
│ dpo_loss                       │ 0.000000       │
│ epoch                          │ 0              │
│ learning_rate                  │ 0.000030       │
│ loss:sum                       │ 0.170094       │
│ margin                         │ 4.731285       │
│ nll_loss                       │ 1.700969       │
│ num_pairs                      │ 32             │
│ num_tokens                     │ 47843          │
│ progress                       │ 0.394737       │
│ rejected_reward                │ -10.992321     │
│ time/get_batch                 │ 0.238663       │
│ time/get_ref_logprobs

tinker_cookbook.utils.ml_log:195 [INFO] 
                     Step 106                      
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┓
┃ Metric                         ┃ Value          ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━┩
│ accuracy                       │ 1.000000       │
│ chosen_reward                  │ 31.502396      │
│ clock_cycle:unique             │ 5017864.000000 │
│ dpo_loss                       │ 0.000501       │
│ epoch                          │ 0              │
│ learning_rate                  │ 0.000030       │
│ loss:sum                       │ 0.186874       │
│ margin                         │ 3.953413       │
│ nll_loss                       │ 1.854266       │
│ num_pairs                      │ 32             │
│ num_tokens                     │ 41868          │
│ progress                       │ 0.398496       │
│ rejected_reward                │ -8.031733      │
│ time/get_batch                 │ 0.326462       │
│ time/get_ref_logprobs

tinker_cookbook.utils.ml_log:195 [INFO] 
                     Step 107                      
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┓
┃ Metric                         ┃ Value          ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━┩
│ accuracy                       │ 1.000000       │
│ chosen_reward                  │ 34.044384      │
│ clock_cycle:unique             │ 5017867.000000 │
│ dpo_loss                       │ 0.000006       │
│ epoch                          │ 0              │
│ learning_rate                  │ 0.000030       │
│ loss:sum                       │ 0.182510       │
│ margin                         │ 4.403194       │
│ nll_loss                       │ 1.824844       │
│ num_pairs                      │ 32             │
│ num_tokens                     │ 42135          │
│ progress                       │ 0.402256       │
│ rejected_reward                │ -9.987558      │
│ time/get_batch                 │ 0.141541       │
│ time/get_ref_logprobs

tinker_cookbook.utils.ml_log:195 [INFO] 
                     Step 108                      
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┓
┃ Metric                         ┃ Value          ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━┩
│ accuracy                       │ 1.000000       │
│ chosen_reward                  │ 37.262188      │
│ clock_cycle:unique             │ 5017871.000000 │
│ dpo_loss                       │ 0.004818       │
│ epoch                          │ 0              │
│ learning_rate                  │ 0.000030       │
│ loss:sum                       │ 0.184851       │
│ margin                         │ 4.630501       │
│ nll_loss                       │ 1.720588       │
│ num_pairs                      │ 32             │
│ num_tokens                     │ 50644          │
│ progress                       │ 0.406015       │
│ rejected_reward                │ -9.042820      │
│ time/get_batch                 │ 0.134864       │
│ time/get_ref_logprobs

tinker_cookbook.utils.ml_log:195 [INFO] 
                     Step 109                      
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┓
┃ Metric                         ┃ Value          ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━┩
│ accuracy                       │ 1.000000       │
│ chosen_reward                  │ 42.031021      │
│ clock_cycle:unique             │ 5017882.000000 │
│ dpo_loss                       │ 0.005878       │
│ epoch                          │ 0              │
│ learning_rate                  │ 0.000030       │
│ loss:sum                       │ 0.188370       │
│ margin                         │ 5.455220       │
│ nll_loss                       │ 1.691932       │
│ num_pairs                      │ 32             │
│ num_tokens                     │ 60925          │
│ progress                       │ 0.409774       │
│ rejected_reward                │ -12.521173     │
│ time/get_batch                 │ 0.143675       │
│ time/get_ref_logprobs

## Post-Training

After training completes, the model weights are saved at:
- `{LOG_PATH}/checkpoints/final/weights/` - Final LoRA weights
- `{LOG_PATH}/checkpoints/final/state/` - Training state (optimizer, etc.)

You can use these weights with the tinker sampling client:
```python
model_path = f"tinker://{run_id}/weights/final"
sampling_client = service_client.create_sampling_client(model_path=model_path)
```

In [ ]:
# Verify training completed and get model path for sampling
checkpoint_file = os.path.join(LOG_PATH, "checkpoints.jsonl")
if os.path.exists(checkpoint_file):
    print("Training completed successfully!")
    print(f"Checkpoints saved to: {LOG_PATH}")
    
    # Read checkpoints and get the final model path
    checkpoints = []
    with open(checkpoint_file, "r") as f:
        for line in f:
            checkpoints.append(json.loads(line))
    
    # Get the last checkpoint with state_path (the final trained model)
    final_checkpoint = None
    for cp in reversed(checkpoints):
        if "state_path" in cp:
            final_checkpoint = cp
            break
    
    if final_checkpoint:
        print(f"\nFinal checkpoint: {final_checkpoint['name']}")
        print(f"Model path for sampling: {final_checkpoint['state_path']}")
        
        # Store for easy access
        MODEL_PATH_FOR_SAMPLING = final_checkpoint['state_path']
        print(f"\nTo sample from this model:")
        print(f"  sampling_client = service_client.create_sampling_client(model_path='{MODEL_PATH_FOR_SAMPLING}')")
else:
    print("No checkpoints found. Training may have failed or not started.")

## Train All Constitutions

To train on all constitutions, you can use the following loop:

In [ ]:
# Uncomment to train all constitutions
# from character.utils import constitutions
#
# for constitution in constitutions:
#     print(f"\n{'='*50}")
#     print(f"Training {MODEL} on {constitution}")
#     print(f"{'='*50}")
#     
#     train_path = f"{DATA_PATH}/dpo/{MODEL}/{constitution}.jsonl"
#     comparison_builder = CharacterDPODataBuilder(train_path=train_path)
#     
#     dataset_builder = DPODatasetBuilderFromComparisons(
#         common_config=common_config,
#         comparison_builder=comparison_builder,
#     )
#     
#     log_path = f"/tmp/character-training/{MODEL}-{constitution}"
#     
#     config = train_dpo.Config(
#         log_path=log_path,
#         model_name=model_name,
#         dataset_builder=dataset_builder,
#         learning_rate=LEARNING_RATE,
#         dpo_beta=DPO_BETA,
#         lora_rank=LORA_RANK,
#     )
#     
#     train_dpo.main(config)